In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget

import importlib
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import ExperimentClass


In [48]:
importlib.reload(ExperimentClass)

manip = ExperimentClass.Experiment("20230321_Ek_FvFm_Sinus", "JTS", diff_xaxis = False)

TypeError: Index does not support mutable operations

In [31]:
manip.keys


Index(['Time', '  E 1', ' E 2', ' E 3', ' E 4', ' E 5', ' E 11', ' E 12',
       ' E 13', ' E 14',
       ...
       ' E 262', ' E 263', ' E 264', ' E 265', ' E 266', ' E 267', ' E 268',
       ' E 269', ' E 270', ' E 271'],
      dtype='object', length=193)

In [39]:
manip.keys.replace(" ","")

AttributeError: 'Index' object has no attribute 'replace'

In [34]:
manip.keys == " E 263"

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,